In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm
from prophet import Prophet
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam
import os
import logging
import warnings
from tqdm import tqdm
import seaborn as sns
from joblib import Parallel, delayed
import time
import matplotlib.dates as mdates
import psutil
from itertools import product
from scipy.stats import rankdata

tqdm.monitor_interval = 0

img_dir = 'ensemble_model_results'
os.makedirs(img_dir, exist_ok=True)

logging.basicConfig(
    filename=f'{img_dir}/classical_models_log.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)
warnings.filterwarnings("ignore")

CONFIG = {
    'forecast_horizon': 12,
    'seasonal_periods': 12,
    'min_data_length': 24,
    'img_dir': img_dir,
    'results_file': f'{img_dir}/classical_model_results.csv',
    'n_jobs': 4,  # Reduced to avoid parallel execution issues
    'outlier_threshold': 3,
    'max_diff': 3,
    'lag_features': list(range(1, 7)),
    'rolling_windows': [3, 12],
    'correlation_threshold': 0.2,
    'batch_size': 1000,
    'ensemble_weight_grid': [0.1, 0.3, 0.5, 0.7, 0.9],
    'des_window': 24,
    'temporal_decay': 0.95,
    'nn_hidden_layers': (50, 25),
    'lstm_units': 50,
    'gru_units': 50,
    'lstm_sequence_length': 6,  # Reduced to ensure sufficient sequences
}

def detect_outliers(series, threshold=CONFIG['outlier_threshold']):
    z_scores = np.abs((series - series.mean()) / series.std())
    outliers = z_scores > threshold
    series_clean = series.where(~outliers, series.mean())
    logger.info(f"Detected {outliers.sum()} outliers in series")
    return series_clean

def validate_input_data(df, required_columns):
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns: {required_columns}")
    if df.index.duplicated().any():
        df = df[~df.index.duplicated(keep='first')]
    if not df.index.is_monotonic_increasing:
        df = df.sort_index()
    if df[required_columns].isnull().sum().any():
        logger.warning(f"Missing values in data: {df[required_columns].isnull().sum().to_dict()}")
        df[required_columns] = df[required_columns].interpolate(method='time').fillna(df[required_columns].mean())
    if np.isinf(df[required_columns]).any().any():
        raise ValueError("Data contains infinite values!")
    if not all(df[required_columns].dtypes.apply(lambda x: np.issubdtype(x, np.number))):
        raise ValueError("Some columns are not numeric!")
    return df

def check_stationarity(series, name):
    max_diff = CONFIG['max_diff']
    series_clean = series.dropna().replace([np.inf, -np.inf], np.nan).dropna()
    if len(series_clean) < 2:
        logger.error(f"{name}: Data too short after cleaning!")
        return 0
    d = 0
    while d <= max_diff:
        result = adfuller(series_clean, maxlag=12, regression='c')
        logger.info(f"ADF Test for {name} (d={d}): Statistic={result[0]:.4f}, p-value={result[1]:.4f}")
        if result[1] < 0.05:
            logger.info(f"{name} stationary at differencing order d={d}")
            return d
        if d == max_diff:
            logger.warning(f"{name} not stationary after {max_diff} differencing. Using d={d}.")
            return d
        series_clean = series_clean.diff().dropna()
        if len(series_clean) < 2:
            logger.warning(f"{name}: Data too short after differencing {d+1}!")
            return d
        d += 1
    return d

def create_features(
    data: pd.DataFrame,
    target: str,
    lags: list = CONFIG['lag_features'],
    rolling_windows: list = CONFIG['rolling_windows'],
    seasonal_features: bool = True
) -> pd.DataFrame:
    logger.info(f"Creating features for {target}, data shape: {data.shape}")
    if len(data) < CONFIG['min_data_length']:
        raise ValueError(f"Data too short: {len(data)} rows")
    if not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be DatetimeIndex")
    exog_var = 'cpi_yoy'
    required_cols = [target, exog_var]
    if not all(col in data.columns for col in required_cols):
        raise ValueError(f"Missing columns: {required_cols}")
    df = data[required_cols].copy()
    df = df.astype(np.float32)
    for col in required_cols:
        for lag in lags:
            df[f'{col}_lag_{lag}'] = df[col].shift(lag, fill_value=df[col].mean())
        for window in rolling_windows:
            df[f'{col}_roll_mean_{window}'] = df[col].rolling(window=window, min_periods=1).mean()
    if seasonal_features:
        period = CONFIG['seasonal_periods']
        df['month_sin'] = np.sin(2 * np.pi * df.index.month / period).astype(np.float32)
        df['month_cos'] = np.cos(2 * np.pi * df.index.month / period).astype(np.float32)
        logger.info("Added seasonal features")
    df['quarter'] = df.index.quarter.astype(np.int8)
    correlations = df.corr()[target].drop(required_cols, errors='ignore')
    low_corr_cols = correlations[abs(correlations) < CONFIG['correlation_threshold']].index
    if low_corr_cols.any():
        logger.info(f"Dropping low-correlation features: {list(low_corr_cols)}")
        df = df.drop(columns=low_corr_cols)
    logger.info(f"Features after processing: {list(df.columns)}")
    return df

def create_lstm_sequences(data, sequence_length):
    if len(data) < sequence_length + 1:
        logger.error(f"Insufficient data for sequences: {len(data)} < {sequence_length + 1}")
        return np.array([]), np.array([])
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

def calculate_metrics(actual, predicted):
    actual = np.array(actual, dtype=np.float32)
    predicted = np.array(predicted, dtype=np.float32)
    valid_mask = ~np.isnan(actual) & ~np.isnan(predicted) & ~np.isinf(actual) & ~np.isinf(predicted)
    actual = actual[valid_mask]
    predicted = predicted[valid_mask]
    if len(actual) == 0:
        logger.warning("No valid data for metrics calculation!")
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    mape = mean_absolute_percentage_error(actual, predicted) * 100 if np.all(np.abs(actual) > 1e-8) else np.nan
    smape = 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))
    norm_mape = mape / np.mean(np.abs(actual)) if not np.isnan(mape) else np.nan
    directional_acc = np.mean((np.diff(actual) * np.diff(predicted)) > 0) * 100 if len(actual) > 1 else np.nan
    return rmse, mae, mape, smape, norm_mape, directional_acc

def plot_decomposition(series, period, filename):
    decomposition = seasonal_decompose(series, period=period, model='additive')
    fig, axes = plt.subplots(4, 1, figsize=(12, 8), sharex=True)
    axes[0].plot(series.index, series, label='Original'); axes[0].legend(loc='upper left')
    axes[1].plot(series.index, decomposition.trend, label='Trend'); axes[1].legend(loc='upper left')
    axes[2].plot(series.index, decomposition.seasonal, label='Seasonal'); axes[2].legend(loc='upper left')
    axes[3].plot(series.index, decomposition.resid, label='Residual'); axes[3].legend(loc='upper left')
    axes[3].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    axes[3].xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.xticks(rotation=45)
    plt.tight_layout()
    try:
        plt.savefig(os.path.join(CONFIG['img_dir'], filename), dpi=150, bbox_inches='tight')
        logger.info(f"Saved decomposition plot: {filename}")
    except Exception as e:
        logger.error(f"Error saving decomposition plot: {str(e)}")
    plt.close()

def plot_forecast(historical, test, forecast, forecast_index, title, ylabel, filename, confidence_intervals=None):
    plt.figure(figsize=(10, 5))
    plt.plot(historical.index, historical, label='Historical', color='blue')
    plt.plot(test.index, test, label='Actual (Test)', color='green')
    plt.plot(forecast_index, forecast, label='Forecast', color='orange', linestyle='--', linewidth=2)
    if confidence_intervals:
        plt.fill_between(forecast_index, confidence_intervals[0], confidence_intervals[1], color='orange', alpha=0.2)
    plt.title(title); plt.xlabel('Time'); plt.ylabel(ylabel); plt.legend(); plt.grid(True)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.xticks(rotation=45)
    plt.tight_layout()
    try:
        plt.savefig(os.path.join(CONFIG['img_dir'], filename), dpi=150, bbox_inches='tight')
        logger.info(f"Saved plot: {filename}")
    except Exception as e:
        logger.error(f"Error saving plot: {str(e)}")
    plt.close()

def plot_comparison_forecasts(historical, test, forecasts, forecast_index, title, ylabel, filename, metrics=None):
    plt.figure(figsize=(12, 6))
    plt.plot(historical.index, historical, label='Historical', color='blue')
    plt.plot(test.index, test, label='Actual (Test)', color='green')
    colors = sns.color_palette("husl", len(forecasts))
    for (model_name, forecast), color in zip(forecasts.items(), colors):
        rmse = metrics.get(model_name, {}).get('RMSE', np.nan) if metrics else np.nan
        if forecast is None or pd.isna(rmse):
            continue
        plt.plot(forecast_index, forecast, label=f'Forecast {model_name} (RMSE: {rmse:.4f})', linestyle='--', color=color)
    plt.title(title); plt.xlabel('Time'); plt.ylabel(ylabel); plt.legend(); plt.grid(True)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.xticks(rotation=45)
    plt.tight_layout()
    try:
        plt.savefig(os.path.join(CONFIG['img_dir'], filename), dpi=150, bbox_inches='tight')
        logger.info(f"Saved comparison plot: {filename}")
    except Exception as e:
        logger.error(f"Error saving comparison plot: {str(e)}")
    plt.close()

def plot_metrics_bar(metrics_df, filename):
    plt.figure(figsize=(14, 10))
    metrics = ['RMSE', 'MAE', 'MAPE', 'sMAPE', 'NormMAPE', 'DirAcc']
    for i, metric in enumerate(metrics, 1):
        plt.subplot(2, 3, i)
        sns.barplot(x='Model', y=metric, data=metrics_df)
        plt.title(f'{metric} Comparison')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
    try:
        plt.savefig(os.path.join(CONFIG['img_dir'], filename), dpi=150, bbox_inches='tight')
        logger.info(f"Saved metrics bar plot: {filename}")
    except Exception as e:
        logger.error(f"Error saving metrics bar plot: {str(e)}")
    plt.close()

def plot_residual_acf(residuals, title, filename):
    if residuals is None or len(residuals) < 2:
        logger.warning(f"Skipping ACF: Insufficient residual data - {title}")
        return
    plt.figure(figsize=(5, 3))
    max_lags = min(12, len(residuals) - 1)
    if max_lags < 1:
        return
    try:
        plot_acf(residuals, lags=max_lags, title=title)
        plt.tight_layout()
        plt.savefig(os.path.join(CONFIG['img_dir'], filename), dpi=150, bbox_inches='tight')
        logger.info(f"Saved ACF plot: {filename}")
    except Exception as e:
        logger.error(f"Error saving ACF plot: {str(e)}")
    plt.close()

def run_exponential_smoothing(train, test, forecast_index, seasonal_periods=CONFIG['seasonal_periods']):
    start_time = time.time()
    try:
        model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=seasonal_periods).fit(
            smoothing_level=0.2, smoothing_slope=0.1, smoothing_seasonal=0.3, optimized=False)
        forecast = model.forecast(CONFIG['forecast_horizon'])
        residuals = train - model.fittedvalues
        forecast = pd.Series(forecast.values, index=forecast_index, dtype=np.float32)
        resid_std = np.std(residuals)
        ci_lower = forecast - 1.96 * resid_std
        ci_upper = forecast + 1.96 * resid_std
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"Exponential Smoothing: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, (ci_lower, ci_upper)
    except Exception as e:
        logger.error(f"Error Exponential Smoothing: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_arima(train, test, forecast_index):
    start_time = time.time()
    try:
        d = check_stationarity(train, "ARIMA")
        model = pm.auto_arima(train, start_p=0, start_q=0, max_p=2, max_q=2, d=d, max_d=1,
                              seasonal=False, stepwise=True, trace=False, error_action='ignore',
                              suppress_warnings=True, information_criterion='aic', maxiter=30)
        forecast = model.predict(n_periods=CONFIG['forecast_horizon'])
        residuals = train - model.predict_in_sample()
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"ARIMA (order={model.order}): RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error ARIMA: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_sarima(train, test, forecast_index, seasonal_periods=CONFIG['seasonal_periods']):
    start_time = time.time()
    try:
        d = check_stationarity(train, "SARIMA")
        model = pm.auto_arima(train, start_p=0, start_q=0, max_p=2, max_q=2, d=d, max_d=1,
                              seasonal=True, m=seasonal_periods, start_P=0, start_Q=0, max_P=1, max_Q=1, max_D=1,
                              stepwise=True, trace=False, error_action='ignore', suppress_warnings=True,
                              information_criterion='aic', maxiter=30)
        forecast = model.predict(n_periods=CONFIG['forecast_horizon'])
        residuals = train - model.predict_in_sample()
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"SARIMA (order={model.order}, seasonal_order={model.seasonal_order}): RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error SARIMA: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_sarimax(train, test, forecast_index, exog_train, exog_test):
    start_time = time.time()
    try:
        exog_train = exog_train.fillna(exog_train.mean())
        exog_test = exog_test.fillna(exog_test.mean())
        if len(exog_train) != len(train) or len(exog_test) != CONFIG['forecast_horizon']:
            raise ValueError(f"Exogenous data size mismatch")
        d = check_stationarity(train, "SARIMAX")
        model = pm.auto_arima(train, exogenous=exog_train, start_p=0, start_q=0, max_p=2, max_q=2, d=d, max_d=1,
                              seasonal=True, m=CONFIG['seasonal_periods'], start_P=0, start_Q=0, max_P=1, max_Q=1, max_D=1,
                              stepwise=True, trace=False, error_action='ignore', suppress_warnings=True,
                              information_criterion='aic', maxiter=30)
        forecast = model.predict(n_periods=CONFIG['forecast_horizon'], exogenous=exog_test)
        residuals = train - model.predict_in_sample(exogenous=exog_train)
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"SARIMAX (order={model.order}, seasonal_order={model.seasonal_order}): RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error SARIMAX: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_prophet(train, test, forecast_index):
    start_time = time.time()
    try:
        df_train = pd.DataFrame({'ds': train.index, 'y': train.values})
        model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False,
                        changepoint_prior_scale=0.05, seasonality_prior_scale=10.0, n_changepoints=10).fit(df_train)
        future = pd.DataFrame({'ds': forecast_index})
        forecast = model.predict(future)
        forecast_series = pd.Series(forecast['yhat'].values, index=forecast_index, dtype=np.float32)
        residuals = train - model.predict(df_train)['yhat']
        ci_lower = pd.Series(forecast['yhat_lower'].values, index=forecast_index, dtype=np.float32)
        ci_upper = pd.Series(forecast['yhat_upper'].values, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast_series)
        logger.info(f"Prophet: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast_series, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, (ci_lower, ci_upper)
    except Exception as e:
        logger.error(f"Error Prophet: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_linear_regression(train, test, forecast_index):
    start_time = time.time()
    try:
        X_train = np.arange(len(train)).reshape(-1, 1).astype(np.float32)
        X_test = np.arange(len(train), len(train) + CONFIG['forecast_horizon']).reshape(-1, 1).astype(np.float32)
        model = LinearRegression(n_jobs=1).fit(X_train, train)
        forecast = model.predict(X_test)
        residuals = train - model.predict(X_train)
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"Linear Regression: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error Linear Regression: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_xgboost(train, test, forecast_index, exog_train, exog_test):
    start_time = time.time()
    try:
        exog_train = exog_train.fillna(exog_train.mean())
        exog_test = exog_test.fillna(exog_test.mean())
        scaler = StandardScaler()
        exog_train_scaled = scaler.fit_transform(exog_train).astype(np.float32)
        exog_test_scaled = scaler.transform(exog_test).astype(np.float32)
        param_grid = {
            'n_estimators': [50],
            'learning_rate': [0.05],
            'max_depth': [3],
            'subsample': [0.8]
        }
        model = GridSearchCV(
            XGBRegressor(random_state=42, tree_method='hist'),
            param_grid,
            cv=TimeSeriesSplit(n_splits=3),
            scoring='neg_mean_squared_error',
            n_jobs=1
        )
        model.fit(exog_train_scaled, train)
        logger.info(f"Best XGBoost params: {model.best_params_}")
        forecast = model.predict(exog_test_scaled)
        residuals = train - model.predict(exog_train_scaled)
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"XGBoost: RMSE={rmse:.4f}, DirAcc={dir_acc:.2f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error XGBoost: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_arima_xgboost_hybrid(train, test, forecast_index, exog_train, exog_test):
    start_time = time.time()
    try:
        d = check_stationarity(train, "ARIMA-XGBoost")
        arima_model = pm.auto_arima(train, start_p=0, start_q=0, max_p=2, max_q=2, d=d, max_d=1,
                                    seasonal=False, stepwise=True, trace=False, error_action='ignore',
                                    suppress_warnings=True, information_criterion='aic', maxiter=30)
        arima_fitted = arima_model.predict_in_sample()
        arima_forecast = arima_model.predict(n_periods=CONFIG['forecast_horizon'])
        residuals = train - arima_fitted
        exog_train = exog_train.fillna(exog_train.mean())
        exog_test = exog_test.fillna(exog_test.mean())
        scaler = StandardScaler()
        exog_train_scaled = scaler.fit_transform(exog_train).astype(np.float32)
        exog_test_scaled = scaler.transform(exog_test).astype(np.float32)
        xgb_model = XGBRegressor(n_estimators=50, learning_rate=0.05, max_depth=3, subsample=0.8,
                                 random_state=42, tree_method='hist')
        xgb_model.fit(exog_train_scaled, residuals)
        residual_forecast = xgb_model.predict(exog_test_scaled)
        forecast = arima_forecast + residual_forecast
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"ARIMA-XGBoost Hybrid: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, None, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error ARIMA-XGBoost Hybrid: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_prophet_nn_hybrid(train, test, forecast_index):
    start_time = time.time()
    try:
        df_train = pd.DataFrame({'ds': train.index, 'y': train.values})
        prophet_model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False,
                                changepoint_prior_scale=0.05, seasonality_prior_scale=10.0, n_changepoints=10).fit(df_train)
        future = pd.DataFrame({'ds': forecast_index})
        prophet_forecast = prophet_model.predict(future)
        prophet_fitted = prophet_model.predict(df_train)
        residuals = train - prophet_fitted['yhat']
        X_train = np.arange(len(train)).reshape(-1, 1).astype(np.float32)
        X_test = np.arange(len(train), len(train) + CONFIG['forecast_horizon']).reshape(-1, 1).astype(np.float32)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        nn_model = MLPRegressor(hidden_layer_sizes=CONFIG['nn_hidden_layers'], activation='relu',
                                solver='adam', max_iter=200, random_state=42)
        nn_model.fit(X_train_scaled, residuals)
        residual_forecast = nn_model.predict(X_test_scaled)
        forecast = prophet_forecast['yhat'].values + residual_forecast
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"Prophet-NN Hybrid: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, None, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error Prophet-NN Hybrid: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_sarima_stacking_hybrid(train, test, forecast_index, exog_train, exog_test):
    start_time = time.time()
    try:
        d = check_stationarity(train, "SARIMA-Stacking")
        sarima_model = pm.auto_arima(train, start_p=0, start_q=0, max_p=2, max_q=2, d=d, max_d=1,
                                     seasonal=True, m=CONFIG['seasonal_periods'], start_P=0, start_Q=0, max_P=1, max_Q=1, max_D=1,
                                     stepwise=True, trace=False, error_action='ignore', suppress_warnings=True,
                                     information_criterion='aic', maxiter=30)
        sarima_fitted = sarima_model.predict_in_sample()
        sarima_forecast = sarima_model.predict(n_periods=CONFIG['forecast_horizon'])
        exog_train = exog_train.fillna(exog_train.mean())
        exog_test = exog_test.fillna(exog_test.mean())
        train_features = exog_train.copy()
        test_features = exog_test.copy()
        train_features['sarima_fitted'] = sarima_fitted
        test_features['sarima_forecast'] = sarima_forecast
        scaler = StandardScaler()
        train_features_scaled = scaler.fit_transform(train_features).astype(np.float32)
        test_features_scaled = scaler.transform(test_features).astype(np.float32)
        xgb_model = XGBRegressor(n_estimators=50, learning_rate=0.05, max_depth=3, subsample=0.8,
                                 random_state=42, tree_method='hist')
        xgb_model.fit(train_features_scaled, train)
        forecast = xgb_model.predict(test_features_scaled)
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"SARIMA-Stacking Hybrid: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, None, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error SARIMA-Stacking Hybrid: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_sarima_lstm_hybrid(train, test, forecast_index):
    start_time = time.time()
    try:
        # Step 1: Fit SARIMA model
        d = check_stationarity(train, "SARIMA-LSTM")
        sarima_model = pm.auto_arima(train, start_p=0, start_q=0, max_p=2, max_q=2, d=d, max_d=1,
                                     seasonal=True, m=CONFIG['seasonal_periods'], start_P=0, start_Q=0, max_P=1, max_Q=1, max_D=1,
                                     stepwise=True, trace=False, error_action='ignore', suppress_warnings=True,
                                     information_criterion='aic', maxiter=30)
        sarima_fitted = sarima_model.predict_in_sample()
        sarima_forecast = sarima_model.predict(n_periods=CONFIG['forecast_horizon'])
        residuals = train - sarima_fitted
        # Step 2: Prepare data for LSTM
        scaler = StandardScaler()
        residuals_scaled = scaler.fit_transform(residuals.values.reshape(-1, 1)).astype(np.float32)
        sequence_length = CONFIG['lstm_sequence_length']
        X_train, y_train = create_lstm_sequences(residuals_scaled, sequence_length)
        if len(X_train) == 0 or len(y_train) == 0:
            logger.error("Insufficient data for LSTM sequences")
            return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None
        # Step 3: Build and train LSTM model
        lstm_model = Sequential([
            LSTM(CONFIG['lstm_units'], input_shape=(sequence_length, 1), return_sequences=False),
            Dense(1)
        ])
        lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        lstm_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
        # Step 4: Forecast residuals
        last_sequence = residuals_scaled[-sequence_length:].reshape(1, sequence_length, 1)
        residual_forecast_scaled = []
        for _ in range(CONFIG['forecast_horizon']):
            pred = lstm_model.predict(last_sequence, verbose=0)
            residual_forecast_scaled.append(pred[0, 0])
            last_sequence = np.roll(last_sequence, -1, axis=1)
            last_sequence[0, -1, 0] = pred[0, 0]
        residual_forecast = scaler.inverse_transform(np.array(residual_forecast_scaled).reshape(-1, 1)).flatten()
        # Step 5: Combine forecasts
        forecast = sarima_forecast + residual_forecast
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"SARIMA-LSTM Hybrid: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, None, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error SARIMA-LSTM Hybrid: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_sarima_gru_hybrid(train, test, forecast_index):
    start_time = time.time()
    try:
        # Step 1: Fit SARIMA model
        d = check_stationarity(train, "SARIMA-GRU")
        sarima_model = pm.auto_arima(train, start_p=0, start_q=0, max_p=2, max_q=2, d=d, max_d=1,
                                     seasonal=True, m=CONFIG['seasonal_periods'], start_P=0, start_Q=0, max_P=1, max_Q=1, max_D=1,
                                     stepwise=True, trace=False, error_action='ignore', suppress_warnings=True,
                                     information_criterion='aic', maxiter=30)
        sarima_fitted = sarima_model.predict_in_sample()
        sarima_forecast = sarima_model.predict(n_periods=CONFIG['forecast_horizon'])
        residuals = train - sarima_fitted
        # Step 2: Prepare data for GRU
        scaler = StandardScaler()
        residuals_scaled = scaler.fit_transform(residuals.values.reshape(-1, 1)).astype(np.float32)
        sequence_length = CONFIG['lstm_sequence_length']
        X_train, y_train = create_lstm_sequences(residuals_scaled, sequence_length)
        if len(X_train) == 0 or len(y_train) == 0:
            logger.error("Insufficient data for GRU sequences")
            return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None
        # Step 3: Build and train GRU model
        gru_model = Sequential([
            GRU(CONFIG['gru_units'], input_shape=(sequence_length, 1), return_sequences=False),
            Dense(1)
        ])
        gru_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        gru_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
        # Step 4: Forecast residuals
        last_sequence = residuals_scaled[-sequence_length:].reshape(1, sequence_length, 1)
        residual_forecast_scaled = []
        for _ in range(CONFIG['forecast_horizon']):
            pred = gru_model.predict(last_sequence, verbose=0)
            residual_forecast_scaled.append(pred[0, 0])
            last_sequence = np.roll(last_sequence, -1, axis=1)
            last_sequence[0, -1, 0] = pred[0, 0]
        residual_forecast = scaler.inverse_transform(np.array(residual_forecast_scaled).reshape(-1, 1)).flatten()
        # Step 5: Combine forecasts
        forecast = sarima_forecast + residual_forecast
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"SARIMA-GRU Hybrid: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, None, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error SARIMA-GRU Hybrid: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_prophet_xgboost_hybrid(train, test, forecast_index, exog_train, exog_test):
    start_time = time.time()
    try:
        df_train = pd.DataFrame({'ds': train.index, 'y': train.values})
        prophet_model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False,
                                changepoint_prior_scale=0.05, seasonality_prior_scale=10.0, n_changepoints=10).fit(df_train)
        future = pd.DataFrame({'ds': forecast_index})
        prophet_forecast = prophet_model.predict(future)
        prophet_fitted = prophet_model.predict(df_train)
        residuals = train - prophet_fitted['yhat']
        exog_train = exog_train.fillna(exog_train.mean())
        exog_test = exog_test.fillna(exog_test.mean())
        scaler = StandardScaler()
        exog_train_scaled = scaler.fit_transform(exog_train).astype(np.float32)
        exog_test_scaled = scaler.transform(exog_test).astype(np.float32)
        xgb_model = XGBRegressor(n_estimators=50, learning_rate=0.05, max_depth=3, subsample=0.8,
                                 random_state=42, tree_method='hist')
        xgb_model.fit(exog_train_scaled, residuals)
        residual_forecast = xgb_model.predict(exog_test_scaled)
        forecast = prophet_forecast['yhat'].values + residual_forecast
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"Prophet-XGBoost Hybrid: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, None, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error Prophet-XGBoost Hybrid: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_ets_gradient_boosting_hybrid(train, test, forecast_index):
    start_time = time.time()
    try:
        ets_model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=CONFIG['seasonal_periods']).fit(
            smoothing_level=0.2, smoothing_slope=0.1, smoothing_seasonal=0.3, optimized=False)
        ets_fitted = ets_model.fittedvalues
        ets_forecast = ets_model.forecast(CONFIG['forecast_horizon'])
        residuals = train - ets_fitted
        X_train = np.arange(len(train)).reshape(-1, 1).astype(np.float32)
        X_test = np.arange(len(train), len(train) + CONFIG['forecast_horizon']).reshape(-1, 1).astype(np.float32)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        gb_model = GradientBoostingRegressor(n_estimators=50, learning_rate=0.05, max_depth=3, random_state=42)
        gb_model.fit(X_train_scaled, residuals)
        residual_forecast = gb_model.predict(X_test_scaled)
        forecast = ets_forecast + residual_forecast
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"ETS-Gradient Boosting Hybrid: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, None, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error ETS-Gradient Boosting Hybrid: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_weighted_ensemble(forecasts, metrics, test, forecast_index):
    start_time = time.time()
    try:
        models = [m for m in forecasts if metrics[m]['RMSE'] > 0 and not pd.isna(metrics[m]['RMSE'])]
        if not models:
            logger.error("No valid models for weighted ensemble")
            return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None
        best_rmse = np.inf
        best_weights = None
        weight_grid = CONFIG['ensemble_weight_grid']
        weight_combinations = list(product(weight_grid, repeat=len(models)))
        for weights in weight_combinations[:100]:
            weights = np.array(weights) / np.sum(weights)
            forecast = pd.Series(0, index=forecast_index, dtype=np.float32)
            for model, weight in zip(models, weights):
                forecast += weight * forecasts[model]
            rmse = np.sqrt(mean_squared_error(test, forecast))
            if rmse < best_rmse:
                best_rmse = rmse
                best_weights = dict(zip(models, weights))
        forecast = pd.Series(0, index=forecast_index, dtype=np.float32)
        for model, weight in best_weights.items():
            forecast += weight * forecasts[model]
        residuals = None
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"Weighted Ensemble (weights={best_weights}): RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error Weighted Ensemble: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_stacking_ensemble(forecasts, metrics, train, test, forecast_index):
    start_time = time.time()
    try:
        models = [m for m in forecasts if metrics[m]['RMSE'] > 0 and not pd.isna(metrics[m]['RMSE'])]
        if not models:
            logger.error("No valid models for stacking ensemble")
            return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None
        train_forecasts = pd.DataFrame({model: forecasts[model].reindex(train.index, method='nearest') for model in models})
        test_forecasts = pd.DataFrame({model: forecasts[model] for model in models})
        train_forecasts = train_forecasts.fillna(train_forecasts.mean())
        test_forecasts = test_forecasts.fillna(test_forecasts.mean())
        meta_model = LinearRegression(n_jobs=1)
        meta_model.fit(train_forecasts, train)
        forecast = meta_model.predict(test_forecasts)
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        residuals = None
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"Stacking Ensemble: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error Stacking Ensemble: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_voting_ensemble(forecasts, metrics, test, forecast_index):
    start_time = time.time()
    try:
        models = [m for m in forecasts if metrics[m]['RMSE'] > 0 and not pd.isna(metrics[m]['RMSE'])]
        if not models:
            logger.error("No valid models for voting ensemble")
            return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None
        forecast_df = pd.DataFrame({model: forecasts[model] for model in models})
        forecast = forecast_df.median(axis=1)
        forecast = pd.Series(forecast, index=forecast_index, dtype=np.float32)
        residuals = None
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"Voting Ensemble (Median): RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error Voting Ensemble: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_bma_ensemble(forecasts, metrics, test, forecast_index):
    start_time = time.time()
    try:
        models = [m for m in forecasts if metrics[m]['RMSE'] > 0 and not pd.isna(metrics[m]['RMSE'])]
        if not models:
            logger.error("No valid models for BMA ensemble")
            return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None
        rmse_values = np.array([metrics[m]['RMSE'] for m in models])
        bic_approx = np.log(rmse_values ** 2 * len(test))
        weights = np.exp(-0.5 * bic_approx)
        weights = weights / np.sum(weights)
        forecast = pd.Series(0, index=forecast_index, dtype=np.float32)
        for model, weight in zip(models, weights):
            forecast += weight * forecasts[model]
        residuals = None
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"BMA Ensemble (weights={dict(zip(models, weights))}): RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error BMA Ensemble: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_des_ensemble(forecasts, metrics, train, test, forecast_index):
    start_time = time.time()
    try:
        models = [m for m in forecasts if metrics[m]['RMSE'] > 0 and not pd.isna(metrics[m]['RMSE'])]
        if not models:
            logger.error("No valid models for DES ensemble")
            return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None
        window = CONFIG['des_window']
        train_forecasts = pd.DataFrame({model: forecasts[model].reindex(train.index, method='nearest') for model in models})
        train_forecasts = train_forecasts.fillna(train_forecasts.mean())
        forecast = pd.Series(0, index=forecast_index, dtype=np.float32)
        for i in range(len(forecast_index)):
            start_idx = max(0, len(train) - window)
            window_data = train[start_idx:]
            window_forecasts = train_forecasts[start_idx:]
            rmses = {}
            for model in models:
                rmse = np.sqrt(mean_squared_error(window_data, window_forecasts[model]))
                rmses[model] = rmse
            top_models = sorted(rmses, key=rmses.get)[:3]
            if not top_models:
                top_models = models
            weights = [1 / rmses[m] for m in top_models]
            weights = np.array(weights) / np.sum(weights)
            forecast[i] = sum(w * forecasts[m][i] for m, w in zip(top_models, weights))
        residuals = None
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"DES Ensemble: RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error DES Ensemble: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_temporal_weighted_ensemble(forecasts, metrics, test, forecast_index):
    start_time = time.time()
    try:
        models = [m for m in forecasts if metrics[m]['RMSE'] > 0 and not pd.isna(metrics[m]['RMSE'])]
        if not models:
            logger.error("No valid models for temporal weighted ensemble")
            return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None
        decay = CONFIG['temporal_decay']
        rmse_values = np.array([metrics[m]['RMSE'] for m in models])
        weights = np.array([decay ** (i / len(test)) / rmse for i, rmse in enumerate(rmse_values)])
        weights = weights / np.sum(weights)
        forecast = pd.Series(0, index=forecast_index, dtype=np.float32)
        for model, weight in zip(models, weights):
            forecast += weight * forecasts[model]
        residuals = None
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"Temporal Weighted Ensemble (weights={dict(zip(models, weights))}): RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error Temporal Weighted Ensemble: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_rank_based_ensemble(forecasts, metrics, test, forecast_index):
    start_time = time.time()
    try:
        models = [m for m in forecasts if metrics[m]['RMSE'] > 0 and not pd.isna(metrics[m]['RMSE'])]
        if not models:
            logger.error("No valid models for rank-based ensemble")
            return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None
        rmse_values = np.array([metrics[m]['RMSE'] for m in models])
        ranks = rankdata(rmse_values, method='average')
        weights = 1 / ranks
        weights = weights / np.sum(weights)
        forecast = pd.Series(0, index=forecast_index, dtype=np.float32)
        for model, weight in zip(models, weights):
            forecast += weight * forecasts[model]
        residuals = None
        rmse, mae, mape, smape, norm_mape, dir_acc = calculate_metrics(test, forecast)
        logger.info(f"Rank-Based Ensemble (weights={dict(zip(models, weights))}): RMSE={rmse:.4f}, Time={time.time() - start_time:.2f}s")
        return forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, None
    except Exception as e:
        logger.error(f"Error Rank-Based Ensemble: {str(e)}")
        return None, None, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, None

def run_model_for_target(target, train, test, forecast_index, model_name, model_func, params):
    logger.info(f"Running {model_name} for {target}")
    start_time = time.time()
    try:
        if model_name in ['SARIMAX', 'XGBoost', 'ARIMA-XGBoost Hybrid', 'SARIMA-Stacking Hybrid', 
                          'Prophet-XGBoost Hybrid']:
            exog_var = 'cpi_yoy'
            feature_cols = [exog_var] + [f'{target}_lag_{lag}' for lag in CONFIG['lag_features']] + \
                           [f'{target}_roll_mean_{w}' for w in CONFIG['rolling_windows']] + \
                           [f'{exog_var}_lag_{lag}' for lag in CONFIG['lag_features']] + \
                           [f'{exog_var}_roll_mean_{w}' for w in CONFIG['rolling_windows']] + \
                           ['month_sin', 'month_cos', 'quarter']
            feature_cols = [col for col in feature_cols if col in train.columns]
            exog_train = train[feature_cols]
            exog_test = pd.DataFrame(index=forecast_index, dtype=np.float32)
            exog_test[exog_var] = test[exog_var].reindex(forecast_index).fillna(train[exog_var].mean())
            for col in feature_cols:
                if col != exog_var:
                    exog_test[col] = train[col].iloc[-1] if col in train.columns else train[exog_var].mean()
            forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, ci = model_func(
                train[target], test[target], forecast_index, exog_train, exog_test, **params
            )
        elif model_name in ['Weighted Ensemble', 'Stacking Ensemble', 'Voting Ensemble', 'BMA Ensemble', 
                           'DES Ensemble', 'Temporal Weighted Ensemble', 'Rank-Based Ensemble']:
            if model_name in ['Stacking Ensemble', 'DES Ensemble']:
                params['train'] = train[target]
                forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, ci = model_func(
                    params['forecasts'], params['metrics'], train[target], test[target], forecast_index
                )
            else:
                forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, ci = model_func(
                    params['forecasts'], params['metrics'], test[target], forecast_index
                )
        else:
            forecast, residuals, rmse, mae, mape, smape, norm_mape, dir_acc, ci = model_func(
                train[target], test[target], forecast_index, **params
            )
        if forecast is None or pd.isna(rmse):
            logger.error(f"{model_name} for {target} failed to produce valid forecast or RMSE")
            return None
        plot_forecast(train[target][-36:], test[target], forecast, forecast_index,
                      f'{model_name} Forecast for {target}', target, f'{target}_{model_name}_forecast.png', ci)
        if residuals is not None:
            plot_residual_acf(residuals.dropna(), f'ACF of Residuals - {model_name} ({target})',
                              f'{target}_{model_name}_acf.png')
        logger.info(f"Completed {model_name} for {target} in {time.time() - start_time:.2f}s")
        return {
            'Target': target,
            'Model': model_name,
            'RMSE': rmse,
            'MAE': mae,
            'MAPE': mape,
            'sMAPE': smape,
            'NormMAPE': norm_mape,
            'DirAcc': dir_acc,
            'Forecast': forecast,
            'Residuals': residuals,
            'CI': ci
        }
    except Exception as e:
        logger.error(f"Error running {model_name} for {target}: {str(e)}")
        return None

def main():
    try:
        data = pd.read_csv('data/analyzed_time_series.csv', parse_dates=['time'], index_col='time')
        data = data.asfreq('MS')
        required_columns = ['cpi_mom', 'cpi_yoy']
        data = validate_input_data(data, required_columns)
        for col in required_columns:
            data[col] = detect_outliers(data[col])
        data_features = create_features(data, 'cpi_mom')
        train_size = len(data) - CONFIG['forecast_horizon']
        train, test = data_features[:train_size], data_features[train_size:]
        forecast_index = pd.date_range(start=test.index[0], periods=CONFIG['forecast_horizon'], freq='MS')
        plot_decomposition(data['cpi_mom'], period=CONFIG['seasonal_periods'], filename='cpi_mom_decomposition.png')
        models = {
            'ARIMA': (run_arima, {}),
            'Exponential Smoothing': (run_exponential_smoothing, {}),
            'Prophet': (run_prophet, {}),
            'SARIMA': (run_sarima, {}),
            'SARIMAX': (run_sarimax, {}),
            'Linear Regression': (run_linear_regression, {}),
            'XGBoost': (run_xgboost, {}),
            'ARIMA-XGBoost Hybrid': (run_arima_xgboost_hybrid, {}),
            'Prophet-NN Hybrid': (run_prophet_nn_hybrid, {}),
            'SARIMA-Stacking Hybrid': (run_sarima_stacking_hybrid, {}),
            'SARIMA-LSTM Hybrid': (run_sarima_lstm_hybrid, {}),
            'Prophet-XGBoost Hybrid': (run_prophet_xgboost_hybrid, {}),
            'ETS-Gradient Boosting Hybrid': (run_ets_gradient_boosting_hybrid, {}),
            'SARIMA-GRU Hybrid': (run_sarima_gru_hybrid, {}),
        }
        results = []
        forecasts_mom = {}
        metrics_mom = {}
        logger.info("Running base and hybrid models for cpi_mom")
        tasks = [delayed(run_model_for_target)('cpi_mom', train, test, forecast_index, model_name, model_func, params)
                 for model_name, (model_func, params) in models.items()]
        model_results = Parallel(n_jobs=CONFIG['n_jobs'], verbose=1, backend='loky')(tasks)
        for result in model_results:
            if result is not None:
                results.append({
                    'Target': result['Target'],
                    'Model': result['Model'],
                    'RMSE': result['RMSE'],
                    'MAE': result['MAE'],
                    'MAPE': result['MAPE'],
                    'sMAPE': result['sMAPE'],
                    'NormMAPE': result['NormMAPE'],
                    'DirAcc': result['DirAcc']
                })
                forecasts_mom[result['Model']] = result['Forecast']
                metrics_mom[result['Model']] = {'RMSE': result['RMSE']}
            else:
                logger.warning(f"Result for a cpi_mom model is None, skipping!")
        if forecasts_mom:
            ensemble_methods = [
                ('Weighted Ensemble', run_weighted_ensemble),
                ('Stacking Ensemble', run_stacking_ensemble),
                ('Voting Ensemble', run_voting_ensemble),
                ('BMA Ensemble', run_bma_ensemble),
                ('DES Ensemble', run_des_ensemble),
                ('Temporal Weighted Ensemble', run_temporal_weighted_ensemble),
                ('Rank-Based Ensemble', run_rank_based_ensemble)
            ]
            for ensemble_name, ensemble_func in ensemble_methods:
                ensemble_result = run_model_for_target(
                    'cpi_mom', train, test, forecast_index, ensemble_name, ensemble_func,
                    {'forecasts': forecasts_mom, 'metrics': metrics_mom}
                )
                if ensemble_result is not None:
                    results.append({
                        'Target': ensemble_result['Target'],
                        'Model': ensemble_result['Model'],
                        'RMSE': ensemble_result['RMSE'],
                        'MAE': ensemble_result['MAE'],
                        'MAPE': ensemble_result['MAPE'],
                        'sMAPE': ensemble_result['sMAPE'],
                        'NormMAPE': ensemble_result['NormMAPE'],
                        'DirAcc': ensemble_result['DirAcc']
                    })
                    forecasts_mom[ensemble_result['Model']] = ensemble_result['Forecast']
                    metrics_mom[ensemble_result['Model']] = {'RMSE': ensemble_result['RMSE']}
        if forecasts_mom:
            plot_comparison_forecasts(train['cpi_mom'][-36:], test['cpi_mom'], forecasts_mom, forecast_index,
                                     'Comparison of Forecasts for cpi_mom', 'cpi_mom', 'cpi_mom_model_comparison.png',
                                     metrics=metrics_mom)
        results_df = pd.DataFrame(results)
        print(results_df)
        results_df.to_csv(CONFIG['results_file'], index=False)
        logger.info(f"Results saved to {CONFIG['results_file']}")
        if not results_df.empty:
            plot_metrics_bar(results_df, 'cpi_mom_metrics_comparison.png')
        if forecasts_mom:
            combined_forecast = pd.DataFrame({'Date': forecast_index})
            for model_name, forecast in forecasts_mom.items():
                combined_forecast[f'{model_name}_cpi_mom'] = forecast
            combined_forecast.to_csv(f'{img_dir}/combined_forecast_cpi_mom.csv', index=False)
            logger.info(f"Combined forecasts saved to {img_dir}/combined_forecast_cpi_mom.csv")
    except Exception as e:
        logger.error(f"Main program error: {str(e)}")
        raise

if __name__ == "__main__":
    main()

c:\Users\ASUS\anaconda3\envs\python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 14 out of 14 | elapsed:   33.0s finished


     Target                         Model      RMSE       MAE      MAPE  \
0   cpi_mom                         ARIMA  0.342722  0.274343  0.273675   
1   cpi_mom         Exponential Smoothing  0.306036  0.261548  0.261149   
2   cpi_mom                       Prophet  0.241324  0.173112  0.172649   
3   cpi_mom                        SARIMA  0.314708  0.239793  0.239107   
4   cpi_mom                       SARIMAX  0.314708  0.239793  0.239107   
5   cpi_mom             Linear Regression  0.318365  0.244097  0.243444   
6   cpi_mom                       XGBoost  0.302830  0.212501  0.211616   
7   cpi_mom          ARIMA-XGBoost Hybrid  0.310293  0.223061  0.222278   
8   cpi_mom            SARIMA-LSTM Hybrid  0.330112  0.251534  0.250820   
9   cpi_mom  ETS-Gradient Boosting Hybrid  0.259320  0.225075  0.224679   
10  cpi_mom             SARIMA-GRU Hybrid  0.317174  0.236952  0.236244   
11  cpi_mom             Weighted Ensemble  0.255814  0.207788  0.207294   
12  cpi_mom             S